Reshape.
Want;
- some properties that I can measure!?!
- some visualisations! (what happens when I reshape?)
- better intuition... need a concrete example to play with
-

## Neighborhoods

Picture I already have. Neighbors and where they go to.


## Connectedness (the dual of neighborhoods?)

What about the graph POV?

## How is reshape like a convolution?

For example, this is what we do when we want to do a convolution. Construct a tensor of patches (examples, X, Y, kernel, kernel) and then reshape it into a (examples x X x Y, kernel x kernel ) matrix.

## SVD

! And what about all the reshaping funny business going on in HSVD and HOSVD?


## Parameter sharing?

Aka, parameter sharing schemes. If we write the reshaped, constructed tensor, and show the receptive field of original parameters.
- are the receptive fields local, which tensor-nets/reshapings give local receptive fields?
- ?
-

This idea is orthogonal to reshaping, reshaping is just a nice way to visualise it?


$$\begin{aligned}
&= \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14} & a_{15} & a_{16} \\
a_{21} & a_{22} & a_{23} & a_{24} & a_{25} & a_{26} \\
a_{31} & a_{32} & a_{33} & a_{34} & a_{35} & a_{36} \\
a_{41} & a_{42} & a_{43} & a_{44} & a_{45} & a_{46} \\
a_{51} & a_{52} & a_{53} & a_{54} & a_{55} & a_{56} \\
a_{61} & a_{62} & a_{63} & a_{64} & a_{65} & a_{66} \\
\end{bmatrix} \\
&\text{(stack by columns. reshape by first indices fastest)}\\
&= \begin{bmatrix}
\begin{bmatrix}
a_{11} &  a_{31} & a_{51}\\
a_{21} & a_{41} & a_{61}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{12} &  a_{32} & a_{52}\\
a_{22} & a_{42} & a_{62}\\
\end{bmatrix}\\
\begin{bmatrix}
a_{13} &  a_{33} & a_{53}\\
a_{23} & a_{43} & a_{63}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{14} &  a_{34} & a_{54}\\
a_{24} & a_{44} & a_{64}\\
\end{bmatrix} \\
\begin{bmatrix}
a_{15} &  a_{35} & a_{55}\\
a_{25} & a_{45} & a_{65}\\
\end{bmatrix} & 
\begin{bmatrix}
a_{16} &  a_{36} & a_{56}\\
a_{26} & a_{46} & a_{66}\\
\end{bmatrix} \\
\end{bmatrix}\end{aligned}$$

Distances are not preserved. Originally $a_{33}$ is one index away from
$\{a_{32},a_{34},a_{23},a_{43}\}$. But after the reshaping, the set of
elements that d=1 are $\{ a_{13},a_{53},a_{43},a_{31},a_{35},a_{34} \}$.
If we map these back into the original matrix, we can see that the
‘range’ of the indicies is speading. More are in each elements
neighbourhood. What does this mean?


Is reshaing a linear op!?
Does it commute, associate, ...